### Входные данные

У вас имеется поток данных (генератор data_stream). Поля это случайные величины - так сделано для упрощения генерации данных. Есть три поля (названы по уровню сложности задания)

### Задание
##### Мотивация:
У вас есть куча временных рядов, вы хотите научиться предсказывать следующее значение по 1000 предыдущим. 1000 признаков окна это слишком много, однако вы решили заменить их 5ю: средним, дисперсией, минимумом, медианой и максимумом. Однако, все эти признаки надо подсчитать, причём хочется уметь это делать быстро (в течение часа)
##### Для каждого поля нужно сделать следующее:

1. Пробежаться по данным окном размера 1000 (окно сдвигается на 1, то есть следующее окно пересекается с предыдущим по 999 элементам).

2. Для каждого окна посчитайте среднее значение поля и его дисперсию. Делайте yield этих значений, получая генератор tuple. 

3. Для каждого окна найдине минимум, медиану и максимум в нём. Делайте yield этих значений, получая генератор tuple. 

Ответом, который нужно будет засабмитить в гугл форму, является среднее значение tuple по получившемуся потоку, округлённое до 2го знака.

### Замечания

1. Обратите внимания как генерируются поля. Постарайтесь понять особенность каждого поля и как это можно использовать. Желательно, чтобы для каждого поля у вас было своё решение, максимально эффективно использующее знание об этом поле.
2. Полезные библиотеки: itertools, numpy, collections + всё что найдёте в интернете и можно поставить через pip install
3. **Медианой отсортированного массива arr считайте значение arr[len(arr) // 2]**



Если измерять время работы функций временем работы функции example, то примерное время работы такое:
Одновременно среднее, дисперсия - 1.17
Одновременно минимум, максимум и медиана:easy - 0.87
medium - 2.11
nightmare - 2.85


#### Генерация данных

In [17]:
from collections import namedtuple
import random
from collections import deque

In [18]:
Record = namedtuple('Record', 'easy medium nightmare')

def data_stream():
    random_generator = random.Random(42)
    easy = 0
    for _ in range(10000000):
        easy += random_generator.randint(0, 2) 
        medium = random_generator.randint(0, 256 - 1)
        nightmare = random_generator.randint(0, 1000000000 - 1)
        
        yield Record(
            easy=easy,
            medium=medium,
            nightmare=nightmare
        )
        
def easy_stream():
    for record in data_stream():
        yield record.easy
        
def medium_stream():
    for record in data_stream():
        yield record.medium
        
def nightmare_stream():
    for record in data_stream():
        yield record.nightmare

#### Подсчёт среднего значения tuple по потоку

In [11]:
import numpy as np

def get_tuple_stream_mean(stream, number_of_values):
    result = np.zeros(number_of_values, dtype='object')
    count = 0. 
    for streamed_tuple in stream:
        result += streamed_tuple
        count += 1
    return ['{:0.2f}'.format(x) for x in result / count]

In [12]:
%%time
def example(stream):
    for value in stream:
        yield (value, value + 10)
        
print(get_tuple_stream_mean(example(medium_stream()), 2))

['127.48', '137.48']
Wall time: 1min 17s


In [31]:
def mean_var_stream(stream):
        for value in stream:
            yield (value, value**2)

In [13]:
#  generator for stream to calculate statistics (mean and var) and yeild them as tuple
def mean_var_rolling_generator(stream, WinSize=1000):
    count = 0
    running_sum = 0.0
    running_sq_sum = 0.0
    #arr = np.empty()
    arr = deque()
    for value in stream:
        count +=1
        arr.append(value)
        running_sum += value
        running_sq_sum += value**2
        if count > (WinSize-1):
            yield( running_sum / WinSize, # rolling mean
                   running_sq_sum / WinSize - (running_sum / WinSize)**2 ) # rolling variance
            tmp_out = arr.popleft() # delete leftmoast elem from arr
            running_sum -= tmp_out
            running_sq_sum -= tmp_out**2

In [19]:
%%time
print( get_tuple_stream_mean(mean_var_rolling_generator(easy_stream()), 2))

['4999675.28', '83441.08']
Wall time: 1min 36s


In [20]:
%%time
print( get_tuple_stream_mean(mean_var_rolling_generator(medium_stream()), 2))

['127.48', '5455.17']
Wall time: 1min 33s


In [21]:
%%time
print( get_tuple_stream_mean(mean_var_rolling_generator(nightmare_stream()), 2))

['499880345.88', '83228908564020544.00']
Wall time: 1min 33s


In [32]:
# check the result: get last window array:
arr_tmp = []
count = 0
for i in nightmare_stream():
    count += 1
    if count > 10000000-1000:
        arr_tmp.append(i)

In [35]:
np.array(arr_tmp).var(ddof=0)

82635010010192640.0

In [42]:
np.array(arr_tmp).mean()

516259214.26999998

In [39]:
# print only last tuple:
count = 0
for j in mean_var_rolling_generator(nightmare_stream()):
    count += 1
    if count == 10000000-1000+1:
        print(j)

(516259214.27, 8.263501001017856e+16)


In [41]:
8.263501001017856*10**16 - np.array(arr_tmp).var(ddof=0)

-14080.0

In [43]:
516259214 - 516259214

0

In [44]:
def easy_stats_generator(stream, WinSize=1000):
    count = 0
    min_tmp = 0.0
    max_tmp = 0.0
    median_tmp = 0.0
    arr = deque()
    for value in stream:
        arr.append(value)
        count += 1
        if  count > (WinSize - 1):
            median_tmp = arr[WinSize//2]
            max_tmp = arr[-1]
            min_tmp = arr.popleft()
            yield(min_tmp, max_tmp, median_tmp)

In [47]:
%%time
print(get_tuple_stream_mean(easy_stats_generator(easy_stream()),3))

['4999175.79', '5000174.76', '4999675.78']
Wall time: 1min 22s


In [57]:
def get_val_position(arr, val):
    for i in range(len(arr)):
        if val < arr[i]:
            return i
        if (val > arr[len(arr)-1]) | (val == arr[len(arr)-1]):
            return len(arr)
        
def deq_insert_order(deq, val):
    order_ind = get_val_position(deq, val)
    deq.rotate(-order_ind)
    deq.appendleft(val)
    deq.rotate(order_ind)
    return deq

def medium_stats_generator(stream, WinSize=1000):
    count = 0
    deq_sorted = deque()
    deq = deque()
    for value in stream:
        count += 1
        if count < WinSize:
            deq.append(value)
        if count == WinSize-1:
            deq.append(value)
            deq_sorted = deque(sorted(deq))
        if count > WinSize-1:
            deq.append(value) # add new value
            deq_sorted = deq_insert_order(deq_sorted, value) # add new value
            yield (deq_sorted[0], deq_sorted[-1], deq_sorted[WinSize//2])
            # yield deq_sorted
            deq_sorted.remove(deq[0]) # remove first value in deq
            deq.popleft() # remove first_val

In [59]:
%%time
print(get_tuple_stream_mean(medium_stats_generator(medium_stream()),3))

['0.02', '254.98', '127.47']
Wall time: 29min 23s


In [60]:
%%time
print(get_tuple_stream_mean(medium_stats_generator(nightmare_stream()),3))

['1016523.37', '999018354.43', '499938182.41']
Wall time: 34min 24s


In [61]:
# change second deque on np array        
def np_insert_order(arr, val):
    arr = np.insert(arr, np.searchsorted(arr,val), val)
    return arr

def np_delete_order(arr, val):
    arr = np.delete( arr, np.searchsorted(arr,val))
    return arr

def medium_stats_generator_2(stream, WinSize=1000):
    count = 0
    #arr_sorted = np.array()
    deq = deque()
    for value in stream:
        count += 1
        if count < WinSize:
            deq.append(value)
        if count == WinSize-1:
            deq.append(value)
            arr_sorted = np.array(sorted(deq))
        if count > WinSize-1:
            deq.append(value) # add new value
            arr_sorted = np_insert_order(arr_sorted, value) # add new value
            yield (arr_sorted[0], arr_sorted[-1], arr_sorted[WinSize//2])
            arr_sorted = np_delete_order(arr_sorted, deq[0]) # remove first value in deq
            deq.popleft() # remove first_val

In [62]:
%%time
print(get_tuple_stream_mean(medium_stats_generator_2(medium_stream()),3))

['0.02', '254.98', '127.47']
Wall time: 5min 34s


In [63]:
%%time
print(get_tuple_stream_mean(medium_stats_generator_2(nightmare_stream()),3))

['1016523.37', '999018354.43', '499938182.41']
Wall time: 5min 44s


In [67]:
! pip install C:/Users/Kirill/Downloads/tqdm-4.19.6-py2.py3-none-any.whl

Processing c:\users\kirill\downloads\tqdm-4.19.6-py2.py3-none-any.whl
